# 范型

## 范型类型

__如何分解以下代码?__
> 如图所示

- 在传统的分解中：
    - 将共同部分分解为一个函数
    - 给不同的部分定义参数
- 这里呢？
    - 函数体相同
    - 只有类型不同

<img src="./asset/generics/maxs.png" width="400" />

__分解__: 将公共部分提取出来，创建一个带有可变部分参数的函数。这里创建类型参数

```rs
fn max<T>(x: T, y: T) -> T {
    if x > y {x} else {y}
}

fn main() {
    let x, y: usize = // ...
    println!("Biggest: {}", max::<usize>(x, y));

    let a, b: f32 = // ...
    println!("Biggest: {}", max::<f32>(a, b));
}
```

 或者: 让编译器根据上下文推断 T 的类型
 
 ```rs
println!("Biggest: {}", max(x, y));
println!("Biggest: {}", max(a, b));
 ```


> **Note**:
类型参数不一定要命名为 T

比如可以这样（但是不推荐）

```rust
fn max<Banana>(x: Banana, y: Banana) -> Banana {
    if x > y {x} else {y}
}
```

> **Note**:
可以有多个类型参数

```rust
fn myFunction<T, R, O>(x: T, y:R) -> O {
    // Do stuff
    // return value of type O
}

__Rust 范型没有运行时开销__

- 我们为每种类型都编写了单独的函数！汇编与我们分解之前编写的代码完全相同
- 因此：代码清理对我们没有任何成本（实际上，考虑到高级语言中更好的代码通常会带来性能成本

如图所示：

<img src="./asset/generics/runtime.png" width="800" />

## 如果无法处理每种类型怎么办？

In [ ]:
fn max<T>(x: <T>, y: <T>) -> T {
    if x > y {x} else {y}
}

实际上，上面的代码并不会编译，报错信息如下

<img src="./asset/generics/max_err.png" width="800" />


### Trait 绑定

- 我们需要将 T 限制为 _可比较类型_，即实现了 `PartialOrd` trait 的类型（该 trait 提供 <、<=、>、>= 运算符）。

    ```rust
    fn max<T: PartialOrd>(x:T, y:T) -> T {
        if x > y {x} else (y)
    }

## 范型和数据结构

### 数据结构也可以是泛型的！

- 之前，实现了 `i32` 类型的 `LinkedList`... 现在让它能够存储任何东西！

如图所示:

<img src="./asset/generics/link-generics.png" width="800" />


In [ ]:
struct Node<T> {
    value: T,
    next: Option<Box<Node<T>>>,
}

struct LinkedList<T> {
    head: Option<Box<Node<T>>>,
    length: usize,
}

你之前见过这个...在 `Option` 和 `Result` 中！

<img src="./asset/generics/option-result.png" width="800" />


### 在范型类型上实现方法

```rust
struct Node<T> {
    value: T,
    next: Option<Box<Node<T>>>,
}

struct LinkedList<T> {
    head: Option<Box<Node<T>>>,
    length: usize,
}

// 第一个 T 代表类型参数
// `LinkedList<T>` 表示为类型<T>正在实现的方法
impl<T> LinkedList<T> {
    fn new() -> LinkedList<T> {
        LinkedList{ head: None, length: 0 }
    }

    pub fn back_mut(&mut self) -> Option<Box<Node<T>>> {
        // some implementation
    }

    pub fn push_back(&mut self, val: T) {
        // some implementation
    }
}
```

编译器通常可以根据上下文推断出你使用的变量的类型

如图所示:

<img src="./asset/generics/tips.png" width="600" />


### 在 _绑定的 `trait`_ 上定义方法是有条件的

> 假设我们想要添加一个 `print()` 方法。我们需要 `T` 具有 `Display`，但即使 `T` 没有 `Display`，我们仍希望其他方法存在。

```rust
impl<T: Display> LinkedList<T> {
    pub fn print(&self) {
        let mut curr = self.front();
        while let Some(node) = curr {
            println!("{}", node.value);
            curr = node.next.as_ref();
        }
    }
}

```

- 这个 `LinkedList<String>` 中有 `print` 方法可用，因为 `String` 实现了 `Display` trait。

    ```rust
    fn main() {

        let mut lst: LinkedList<String> = LinkedList::new();
        lst.push_back("Hello world!".to_string());
        lst.print();
    }
    ```

- 然而这样是行不通的。假设 `MyType` 没有实现 `display` trait，一个 `LinkedList<MyType>` 不能调用 `print` 方法。
 
    ```rust
    fn main() {
        let mut lst : LinkedList<MyType> = LinkedList::new();
        lst.push_back(MyType {});
        lst.print();
    }
    ```